In [4]:
import os
import sys
import time

sys.path.append(r'/home/jovyan/thrift/XRpc/services/gen-py') # this is the interface to application
sys.path.append(r'/home/jovyan/thrift/XCoreModelingServer/gen-py') # this is the interface to modeler
#sys.path.append(r'C:\Users\anderegg\Documents\dev\sim4life\ThirdPartyLibs\svn-esx_SuperMash_libraries_thrift_thrift-0.11.0-msvc-14.0\thrift\lib\py\build\lib.win-amd64-2.7') # this is the thrift library

import application # this ithe API to s4l!
from application import Application
from application import ProcessFactory
from application import Logger
from application.ttypes import SessionError

import modeler
from modeler import Modeler

from thrift import Thrift
from thrift.transport import TSocket
from thrift.transport import TTransport
from thrift.protocol import TBinaryProtocol
from thrift.protocol import TMultiplexedProtocol


In [6]:
class XRpcThriftInterface:
    applicationClient = 0
    modelerClient = 0
    studiesClient = 0
    loggerClient = 0
    
    # this is used for standard buffer transports
    def connectToStandardBufferInterface(self, ip, port, clientService):
        #make socket for processFactory interface
        transport = TSocket.TSocket(ip, port)
        # Buffering is critical. Raw sockets are very slow
        transport = TTransport.TBufferedTransport(transport)

        # Wrap in a protocol
        protocol = TBinaryProtocol.TBinaryProtocol(transport)
        # Create a client
        client = clientService.Client(protocol)
        # Connect
        transport.open()
        return client;

    # this is used for multiplexed interfaces (seems not available in nodeJS yet)
    def connectToMultiplexedInterface(self, ip, port, serviceNames, serviceClients):
        #make socket for modeler interface
        socket = TSocket.TSocket(ip, port)
        # Buffering is critical. Raw sockets are very slow
        transport = TTransport.TBufferedTransport(socket)
        # Wrap in a protocol
        protocol = TBinaryProtocol.TBinaryProtocol(transport)

        clients = []
        for i in range(0,len(serviceNames)):
            # Wrap in a protocol for the logger service (it-s a multiplexed protocol...)
            multiplexedProtocol = TMultiplexedProtocol.TMultiplexedProtocol(protocol, serviceNames[i])
            # Create the loggger client
            clients.append(serviceClients[i].Client(multiplexedProtocol))

        # Connect
        transport.open()
        return clients;

    def connectToThriftInterface(self, ip, basePort, useMultiplexedInterface):
        if useMultiplexedInterface == False:
            self.applicationClient = self.connectToStandardBufferInterface(ip, basePort, application.Application)
            self.modelerClient = self.connectToStandardBufferInterface(ip, basePort + 1, modeler.Modeler)
        else:
            serviceNames = ['Application', 'Logger', 'Studies', 'Modeler']
            serviceClients = [application.Application, application.Logger, studies.Studies, modeler.Modeler]
            clients = self.connectToMultiplexedInterface(ip, basePort, serviceNames, serviceClients)
            self.applicationClient = clients[0]
            self.loggerClient = clients[1]
            self.studiesClient = clients[2]
            self.modelerClient = clients[3]
    
    def __init__(self, ip, basePort, multiplexed):
        self.connectToThriftInterface(ip, basePort, multiplexed)
            

In [7]:
IP = 'localhost'
PORT = 9095;
interface = XRpcThriftInterface (IP, PORT, False)

Could not connect to any of [('127.0.0.1', 9095), ('127.0.0.1', 9095)]


TTransportException: Could not connect to any of [('127.0.0.1', 9095), ('127.0.0.1', 9095)]

In [14]:
applicationApiVersion = interface.applicationClient.GetApiVersion()
print ("Getting application version:", applicationApiVersion, "\n")

('Getting application version:', ApiVersion(major=1, minor=0), '\n')


In [15]:
interface.applicationClient.NewDocument()
interface.modelerClient.ImportModel(r'c:\app\data\DemoDec_OnlyHead_Clean_Neuron.smash')
#interface.modelerClient.ImportModel(r'C:\tmp\DecDemo_Modeler_nobones_1group.smash')

In [16]:
interface.modelerClient.GetFilteredEntities(modeler.ttypes.EntityFilterType.SOLID_BODY)

[Entity(name=u'Cover Loops 33', pathUuids=u'00000000-0000-0000-0000-000000000000/a27246d2-11b6-4ee9-8d55-aed7ddc3a564/fa072129-f9b7-452b-8b1a-ec3e85cbed3c', has_geometry=True, color=Color(a=1.0, r=0.9750000238418579, b=0.02499997615814209, g=0.07697167992591858), pathNames=u'Model/Surfaces/Cover Loops 33', children=[], uuid=u'fa072129-f9b7-452b-8b1a-ec3e85cbed3c'),
 Entity(name=u'Cover Loops 23', pathUuids=u'00000000-0000-0000-0000-000000000000/a27246d2-11b6-4ee9-8d55-aed7ddc3a564/3fd40219-0afa-41d6-9649-fb839310e0ad', has_geometry=True, color=Color(a=1.0, r=0.4350651502609253, b=0.9750000238418579, g=0.02499997615814209), pathNames=u'Model/Surfaces/Cover Loops 23', children=[], uuid=u'3fd40219-0afa-41d6-9649-fb839310e0ad'),
 Entity(name=u'Cover Loops 13', pathUuids=u'00000000-0000-0000-0000-000000000000/a27246d2-11b6-4ee9-8d55-aed7ddc3a564/66f27a8f-50e2-4bab-af21-31f6268ffabe', has_geometry=True, color=Color(a=1.0, r=0.02499997615814209, b=0.9750000238418579, g=0.6428734660148621), pa

In [18]:
interface.modelerClient.GetEntitiesEncodedScene(list_of_uuids=[u'd3229400-3d6e-4c9e-b8fb-f0d3994f11ec'], format=modeler.ttypes.SceneFileFormat.GLB)

TApplicationException: TException - service has thrown: ModelerException(message=Could not find an exporter compatible with the given list of entities)

In [18]:
interface.applicationClient.Quit()